In [1]:
%pip install arxiv


Note: you may need to restart the kernel to use updated packages.


In [12]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [14]:
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [15]:
wiki.name

'wikipedia'

In [3]:

import os
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

# Set your API key here
groq_api_key = os.getenv("GROQ_API_KEY")
os.environ['GOOGLE_API_KEY'] = os.getenv("GOOGLE_API_KEY")

llm = ChatGroq(groq_api_key=groq_api_key,model_name="gemma2-9b-it", temperature= 0)

# Load documents from the web
loader = WebBaseLoader("https://docs.smith.langchain.com/")
docs= loader.load()

# Split documents into chunks
documents=RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200).split_documents(docs)


# Create embeddings instance
embeddings=GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# Create FAISS vector store from split documents
vectordb = FAISS.from_documents(documents, embeddings())
retriever = vectordb.as_retriever()
retriever

python-dotenv could not parse statement starting at line 1


TypeError: 'GoogleGenerativeAIEmbeddings' object is not callable

In [ ]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retriever, "langsmith_search",
                                       "Search for information about LangSmith. For any questions about Langsmith, you must use this tool.")


In [9]:
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools.arxiv import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [ ]:
tools=[wiki, arxiv, retriever_tool]

In [ ]:
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("lc_groq/chat-zero-shot-react-description")
prompt.messages

In [4]:
# Agents
from langchain.agents import initialize_agent, AgentType,load_tools
agent = initialize_agent(llm, tools, prompt)

NameError: name 'tools' is not defined

In [ ]:
# Agent Executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent = agent, tools = tools, verbose = True)
agent_executor

In [ ]:
agent_executor.invoke({"input": "What is LangSmith?"})